In [131]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [132]:
doc_1 = "os componentes destacados são os quatro componentes principais como Gateway, Updater, Storage Repo e Argus. Gateway é uma estrutura de processo de fluxo leve. Ele é responsável por converter os diferentes eventos de atualização de fontes upstream para o formato do atualizador de indexação de anúncios e, em seguida, enviá-los ao Updater via Kafka ou RPC direto. Updater é o componente de ingestão de dados, que recebe atualizações do Gateway e as grava no Storage Repo. O repositório de armazenamento mantém todos os dados durante o ciclo de vida do processo do documento do anúncio, incluindo atualizações brutas, dados intermediários e documentos finais de serviço. Argus são serviços de processo de dados acionados por notificação. Depois de ser notificado por um evento de alteração de dados do Repo, um trabalhador Argus lê todos os dados dependentes do repositório de dados e conduz um processo computacional pesado para gerar os documentos servíveis finais. Gateway, Updater e Argus são todos serviços sem estado, feitos para serem facilmente escalados para cima e para baixo."
doc_2 = "Comentários Gerais : Existe uma separação clara entre os mundos tempo-real e não-tempo-real. Os acessos ao mundo tempo-real são feitos via snapshots dos BD de armazenamento geral e de anúncios. Update e Argus são os únicos sistemas a atualizar o armazenamento geral.  Argus usa dados de sinais orgânicos para atualizar o armazenamento geral. Argus serve dados necessários para o mundo tempo-real. Comentários sobre a arquitetura :Gosto da decisão arquitetural de separar os acessos do mundos de tempo-real e não-tempo real. Gosto da boa a decisão de usar o S3 para disparar workflows baseados em filas (kafka), principalmente se está dentro do AWS. Gosto da decisão dos workflows usarem RPCs para disparar as atualizações no serviços. Precisaria refletir mais/ter mais informação para ter uma opinião com relação aos sistemas que serve tempo-real ser atualizado por filas kafka (isso deve ter a ver com controle de fluxo/escala e custo). Comentários sobre a notação : A notação conseguiu exprimir de forma concisa relacionamentos e tipos de sistemas bem complexos"
doc_3 = "É possível notar que há basicamente três tipos de componentes nesse diagrama: services, workflows e datasets. As arestas indicam o fluxo das ações tomadas pelo sistema passando por todos os nós do diagrama. O padrão da flecha também ajuda a compreender como se dá essa comunicação: RPC, Kafka ou S3."
doc_4 = "Os componentes destacados são os quatro componentes principais como Gateway, Updater, Storage Repo e Argus. Gateway é uma estrutura de processo de fluxo leve. Updater é o componente de ingestão de dados, que recebe atualizações do Gateway e as grava no Storage Repo. O repositório de armazenamento mantém todos os dados durante o ciclo de vida do processo do documento do anúncio, incluindo atualizações brutas, dados intermediários e documentos finais de serviço. Argus são serviços de processo de dados acionados por notificação. Depois de ser notificado por um evento de alteração de dados do Repo."
doc_5 = "Dentro desse diagrama é possível extrair informações relativas a bancos de cache, sincronização de databases por meio de eventos (RabbitMQ, Kafka) e interações entre os serviços (Quando atualizar o cache, quando recuperar informações, de onde recuperar, pra quem requisitar, etc)."
corpus = np.array([doc_1, doc_2, doc_3, doc_4, doc_5])

### 1) Aplicar lowercase e remover caracteres

In [133]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [134]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['my phone is not good' 'my phone is amazing']


### 2) Remover stopwords

In [135]:
from nltk.corpus import stopwords
def clear_stopwords(text, language):
    filtered_text = []
    for t in text:
        words = [w for w in t.split() if not w in stopwords.words(language)]
        words = ' '.join(words)
        filtered_text.append(words)
    return np.array(filtered_text)   

In [136]:
corpus_filtered = clear_stopwords(corpus_clear, "portuguese")
print(corpus_filtered)

['my phone is not good' 'my phone is amazing']


### Usando a biblioteca sklearn

In [124]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorize = CountVectorizer()
vector_matrix = vectorize.fit_transform(corpus_filtered)

tokens = vectorize.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)



,ajudou,alimenta,alimentar,apache,armazenamento,armazenar,arquitetura,atendidas,atendimento,através,...,trabalhar,três,usado,vai,valida,verifica,vez,várias,zonas,últimas
text_1,1,0,0,1,2,1,0,1,0,0,...,0,1,1,0,1,1,0,1,1,1
text_2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,3,0,0,0,0,0,0
text_3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
text_4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
text_5,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['text_1','text_2','text_3','text_4', 'text_5'])

,text_1,text_2,text_3,text_4,text_5
text_1,1.000000,0.276790,0.130682,0.029934,0.190500
text_2,0.276790,1.000000,0.196722,0.000000,0.344124
text_3,0.130682,0.196722,1.000000,0.000000,0.114332
text_4,0.029934,0.000000,0.000000,1.000000,0.000000
text_5,0.190500,0.344124,0.114332,0.000000,1.000000
